In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
pip install tensorflow==2.0.0

In [0]:
pip install -r '/content/drive/My Drive/Colab Notebooks/RL-CartPole Project/requirements.txt'

  Using cached https://files.pythonhosted.org/packages/3b/72/e6e483e2db953c11efa44ee21c5fdb6505c4dffa447b4263ca8af6676b62/absl-py-0.8.1.tar.gz
  Using cached https://files.pythonhosted.org/packages/d1/4f/950dfae467b384fc96bc6469de25d832534f6b4441033c39f914efd13418/astor-0.8.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/45/0b/38b06fd9b92dc2b68d58b75f900e97884c45bedd2ff83203d933cf5851c9/future-0.18.2.tar.gz
  Using cached https://files.pythonhosted.org/packages/0e/4f/e9e84e4600c43cae7ce58489c6e73ff4c864557bc4d4d0f0029c79e07f31/grpcio-1.16.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/60/06/cafdd44889200e5438b897388f3075b52a8ef01f28a17366d91de0fa2d05/h5py-2.10.0-cp36-cp36m-manylinux1_x86_64.whl
ERROR: Could not find a version that satisfies the requirement mkl-fft==1.0.15 (from -r /content/drive/My Drive/Colab Notebooks/RL-CartPole Project/requirements.txt (line 14)) (from versions: 1.0.0.17, 1.0.2, 1.0.6)
ERROR: 

In [1]:
import tensorflow as tf
import numpy as np
import gym
import yaml
import os

In [0]:
def model_v1():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units = 32, input_dim = 4, activation = 'relu', dtype = tf.float32))
    model.add(tf.keras.layers.Dense(units = 2, activation = 'softmax', dtype = tf.float32))
    return model

def model_v2():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units = 32, input_dim = 4, activation = 'relu', dtype = tf.float32))
    model.add(tf.keras.layers.Dense(units = 32, input_dim = 4, activation = 'relu', dtype = tf.float32))
    model.add(tf.keras.layers.Dense(units = 32, input_dim = 4, activation = 'relu', dtype = tf.float32))
    model.add(tf.keras.layers.Dense(units = 2, activation = 'softmax', dtype = tf.float32))
    return model

def model_v3():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units = 32, input_dim = 4, activation = 'relu', dtype = tf.float32))
    model.add(tf.keras.layers.Dense(units = 32, input_dim = 32, activation = 'relu', dtype = tf.float32))
    model.add(tf.keras.layers.Dense(units = 32, input_dim = 32, activation = 'relu', dtype = tf.float32))
    model.add(tf.keras.layers.Dense(units = 32, input_dim = 32, activation = 'relu', dtype = tf.float32))
    model.add(tf.keras.layers.Dense(units = 32, input_dim = 32, activation = 'relu', dtype = tf.float32))
    model.add(tf.keras.layers.Dense(units = 2, activation = 'softmax', dtype = tf.float32))
    return model

def model_v4():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units = 32, input_dim = 4, activation = 'relu', dtype = tf.float32))
    model.add(tf.keras.layers.Dense(units = 32, input_dim = 4, activation = 'relu', dtype = tf.float32))
    model.add(tf.keras.layers.Dense(units = 32, input_dim = 4, activation = 'relu', dtype = tf.float32))
    model.add(tf.keras.layers.Dense(units = 32, input_dim = 4, activation = 'relu', dtype = tf.float32))
    model.add(tf.keras.layers.Dense(units = 32, input_dim = 4, activation = 'relu', dtype = tf.float32))
    model.add(tf.keras.layers.Dense(units = 2, activation = 'softmax', dtype = tf.float32))
    return model

def model_v5():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units = 32, input_dim = 4, activation = 'relu', dtype = tf.float32))
    model.add(tf.keras.layers.Dropout(rate = 0.2))
    model.add(tf.keras.layers.Dense(units = 32, input_dim = 32, activation = 'relu', dtype = tf.float32))
    model.add(tf.keras.layers.Dropout(rate = 0.2))
    model.add(tf.keras.layers.Dense(units = 32, input_dim = 32, activation = 'relu', dtype = tf.float32))
    model.add(tf.keras.layers.Dropout(rate = 0.2))
    model.add(tf.keras.layers.Dense(units = 32, input_dim = 32, activation = 'relu', dtype = tf.float32))
    model.add(tf.keras.layers.Dropout(rate = 0.2))
    model.add(tf.keras.layers.Dense(units = 32, input_dim = 32, activation = 'relu', dtype = tf.float32))
    model.add(tf.keras.layers.Dense(units = 2, activation = 'softmax', dtype = tf.float32))
    return model

In [0]:
def discount_rewards(rewards, discount_rate = 0.8):
  """
  Takes in rewards and applies discount_rate

  Reward = Reward[t=0] * pow(discount_rate, 0) + ... + Reward[t=n] * pow(discount_rate, n) 

  :param rewards: numpy.array. The list of rewards to be discounted.
  :param discount_rate: float. Determines the impact of future actions on current reward.
      Valid entries in (0, 1) where a larger discount_rate forces model to consider current actions
      to have smaller effects on future rewards.
  :return: discounted_rewards: numpy.ndarray. Discounted rewards.
  """

  # input checks
  if type(rewards) != np.ndarray:
    raise TypeError('rewards must be of type numpy.array')
  if type(discount_rate) != float:
    raise TypeError('discount_rate must be of type float')
  if not 0 < discount_rate < 1:
    raise ValueError('discount_rate must be in (0, 1)')

  # applying discount in fashion described above
  discounted_rewards = np.zeros_like(a = rewards)
  running_total = 0
  for i in reversed(range(0, rewards.size)):
    running_total = running_total * discount_rate + rewards[i]
    discounted_rewards[i] = running_total
  return discounted_rewards

In [0]:
def train_model(num_episodes, model_version, discount_rate, learning_rate):
    """
    Trains a policy model

    :param num_epochs: int. Determines how many epochs to train for.
    :param model_version: str. Provides the model architecture to use.
    :param discount_rate: float. Determines the impact of future actions on current reward.
      Valid entries in (0, 1) where a larger discount_rate forces model to consider current actions
      to have smaller effects on future rewards.
    :param learning_rate: float. Determines how far of a step is taken along gradient when training.
    :return: None
    """

    # getting available models
    with open('/content/drive/My Drive/Colab Notebooks/RL-CartPole Project/model_architectures.yaml', 'r') as file:
        available_models = yaml.safe_load(stream = file)
        file.close()

     # input checks
    if type(num_episodes) != int:
        raise TypeError('num_episodes must be of type int')
    if num_episodes <= 0:
        raise ValueError('num_episodes must be greater than zero')
    if type(model_version) != str:
        raise TypeError('model_version must be of type string')
    if model_version not in available_models:
        raise ValueError('model_version not available')
    if type(discount_rate) != int and type(discount_rate) != float:
        raise TypeError('discount rate must be of type int or float')
    if discount_rate <= 0:
        raise ValueError('discount_rate must be greater than zero')
    if type(learning_rate) != float:
        raise TypeError('learning_rate must be of type float')
    if learning_rate <= 0 or learning_rate >= 1:
        raise ValueError('learning_rate must be within (0, 1)')

    # determining model sub-version
    model_sub_version_write = False
    sub_version = 0
    while not model_sub_version_write:
        if not os.path.exists(os.path.join('/content/drive/My Drive/Colab Notebooks/RL-CartPole Project/','trained_models', model_version, model_version + '.' + str(sub_version) + '.0')):
            model_sub_version_write = True
            break
        sub_version += 1

    # building model
    model = eval(available_models[model_version])()
    optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
    compute_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)

    # holding gradients
    gradient_holder = model.trainable_variables
    for i, gradient in enumerate(gradient_holder):
        gradient_holder[i] = gradient * 0

    # creating gym environment
    env = gym.make('CartPole-v1')
    env._max_episode_steps = 15000

    scores = []
    every_update = 5

    # training loop
    for episode in range(num_episodes + 1):
        observation = env.reset()

        episode_memory = []
        episode_score = 0
        done = False

        while not done:
            # creates vector of form [Position, Velocity, Angle, Angular Velocity]
            observation = observation.reshape([1, 4])

            with tf.GradientTape() as tape:

                # creating loss function and action
                logits = model(observation)
                a_dist = logits.numpy()
                action = np.random.choice(a = a_dist[0], p = a_dist[0])
                action = np.argmax(a_dist == action)
                loss = compute_loss([action], logits)

            # performing action and getting feedback from environment
            observation, reward, done, info = env.step(action)

            episode_score += reward

            # trick for quicker convergence
            if done:
                reward -= 10

            # getting gradients
            gradients = tape.gradient(target = loss, sources = model.trainable_variables)
            episode_memory.append([gradients, reward])

        scores.append(episode_score)

        # discounting rewards
        episode_memory = np.array(episode_memory)
        episode_memory[:, 1] = discount_rewards(rewards = episode_memory[:, 1], discount_rate = discount_rate)

        # applying rewards to corresponding gradients
        for grads, reward in episode_memory:
            for i, grad in enumerate(grads):
                gradient_holder[i] += grad * reward

        # back-propogating gradients, resetting gradients
        if episode % every_update == 0:
            optimizer.apply_gradients(zip(gradient_holder, model.trainable_variables))
            for i, grad in enumerate(gradient_holder):
                gradient_holder[i] = grad * 0

        # printing progress, saving model
        if episode % 100 == 0:
            print('Episode {} Score {}'.format(episode, np.mean(scores[-20:])))
            tf.keras.models.save_model(model = model,
                                       filepath = os.path.join('/content/drive/My Drive/Colab Notebooks/RL-CartPole Project/',
                                                               'trained_models', model_version, model_version + '.' +
                                                               str(sub_version) +'.{}'.format(episode)))

            tf.keras.models.save_model(model = model,
                                       filepath = os.path.join('/content/',
                                                               'trained_models', model_version, model_version + '.' +
                                                               str(sub_version) +'.{}'.format(episode)))

    final_performance = int(round(np.mean(scores[-20:])))

    # dumping training results into yaml file
    yaml_dump = {}
    yaml_dump['Model Version'] = model_version
    yaml_dump['Model Sub-version'] = sub_version
    yaml_dump['Number of Training Episodes'] = num_episodes
    yaml_dump['Discount Rate'] = discount_rate
    yaml_dump['Learning Rate'] = learning_rate
    yaml_dump['Final Performance'] = final_performance

    # dumping training details into yaml file
    with open(os.path.join('/content/drive/My Drive/Colab Notebooks/RL-CartPole Project/', 'trained_models', model_version, 'model_' + model_version + '.' + str(sub_version) + '_training_details.yaml'), 'w') as file:
        yaml.dump(data = yaml_dump, stream = file)
        file.close()
    with open(os.path.join('/content/trained_models', model_version + '.' + str(sub_version) + '_training_details.yaml'), 'w') as file:
        yaml.dump(data = yaml_dump, stream = file)
        file.close()
        

In [0]:
num_episodes = 1500
model_version = 'v5'
discount_rate = 0.8
learning_rate = 0.01

train_model(num_episodes = num_episodes, model_version = model_version,
                discount_rate = discount_rate, learning_rate = learning_rate)


/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Episode 0 Score 18.0
INFO:tensorflow:Assets written to: /content/drive/My Drive/Colab Notebooks/RL-CartPole Project/trained_models/v5/v5.0.0/assets
INFO:tensorflow:Assets written to: /content/trained_models/v5/v5.0.0/assets
Episode 100 Score 11.35
INFO:tensorflow:Assets written to: /content/drive/My Drive/Colab Notebooks/RL-CartPole Project/trained_models/v5/v5.0.100/assets
INFO:tensorflow:Assets written to: /content/trained_models/v5/v5.0.100/assets
Episode 200 Score 62.5
INFO:tensorflow:Assets written to: /content/drive/My Drive/Colab Notebooks/RL-CartPole Project/trained_models/v5/v5.0.200/assets
INFO:tensorflow:Assets written to: /content/trained_models/v5/v5.0.200/assets
Episode 300 Score 42.95
INFO:tensorflow:Assets written to: /content/drive/My Drive/Colab Notebooks/RL-CartPole Project/trained_models/v5/v5.0.300/assets
INFO:tensorflow:Assets written to: /content/trained_models/v5/v5.0.300/assets
Episode 400 Score 96.75
INFO:tensorflow:Assets written to: /content/drive/My Drive/C